# Install required packages

In [56]:
%pip install -U langchain
%pip install --no-cache-dir sentence-transformers
%pip install -U langchain-community
%pip install langchain-groq

  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
Using cached h11-0.16.0-py3-none-any.whl (37 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Not uninstalling h11 at c:\python312\lib\site-packages, outside environment c:\Users\Admin\OneDrive\Desktop\rag-query-system\ragenv
    Can't uninstall 'h11'. No files were found to uninstall.
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
Using cached h11-0.16.0-py3-none-any.whl (37 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Not uninstalling h11 at c:\python312\lib\site-packages, outside environment c:\Users\Admin\OneDrive\Desktop\rag-query-system\ragenv
    Can't uninstall 'h11'. No files were found to uninstall.
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
Using cached h11-0.16.0-py3-none-any.whl (37 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Not uninstalling h11 at c:\python312\lib\site-packages, outside environment c:\Users\Admin\OneDrive\Desktop\rag-query-system\ragenv
    Can't uninstall 'h11'. No files were found to uninstall.
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Imports

In [57]:

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
import os
import chromadb
import chromadb.config


# Create Embeddings

In [58]:
from sentence_transformers import SentenceTransformer
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
print("Model loaded successfully!")


Model loaded successfully!


In [59]:
import pickle

with open("../data/chunks.pkl", "rb") as f:
    chunks = pickle.load(f)

print(f"✅ Loaded {len(chunks)} chunks!")


✅ Loaded 906 chunks!


# Create Vector Store with ChromaDB

In [60]:
# Store in a folder
persist_directory = "../chroma"

# Create vector store
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory=persist_directory
)

# Save to disk
#vectordb.persist()....//not needed, as ChromaDB handles persistence automatically
print("Embeddings saved to ChromaDB.")


Embeddings saved to ChromaDB.


# Create a retriever

In [61]:
retriever = vectordb.as_retriever()


# Set Up our LLM

In [63]:
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
import shutil

# Load environment variables from .env
load_dotenv()

# Fetch variables from environment
groq_api_key = os.getenv("GROQ_API_KEY")
groq_model = os.getenv("GROQ_MODEL")

if not groq_api_key or not groq_model:
    raise ValueError("GROQ_API_KEY or GROQ_MODEL not found in environment variables.")

# Load retriever from Chroma (already created in Day 3)
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb = Chroma(persist_directory="../chroma", embedding_function=embedding_model)

# Delete everything
#ectordb.delete_collection()

retriever = vectordb.as_retriever()

# Connect to Groq via LangChain
llm = ChatGroq(
    temperature=0,
    api_key = groq_api_key,
    model = groq_model
)

# Build RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# Ask a question
query = "What is chapter one about?"
response = qa_chain.invoke(query)

print("\n Answer:", response["result"])

# source documents used
#for i, doc in enumerate(response["source_documents"]):
    #print(f"\n📄 Source {i+1}:\n", doc.page_content[:300], "...")




 Answer: Chapter one of the story is about the narrator, Margaret Douglas, describing her current state of being tired and weary. She mentions that she has been in the nunnery of the holy St. Brigida of Cheverney for two years, which suggests that she is a nun. The chapter also touches on her past, mentioning a man named Laurence, who is significantly older than her, and her childhood friend Sholto.
